In [1]:
from sklearn.datasets import make_classification
from collections import Counter
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc

In [2]:
X, y = make_classification(n_samples=1000, n_classes=2, random_state=7)

In [3]:
def porc_part(y):
  counter = Counter(y)

  for k,v in counter.items():
    per = v / len(y) * 100
    print('Class=%d, Count=%d, Percentage=%.3f%%' % (k, v, per))



In [ ]:
porc_part(y)

In [5]:
trainX, valX, trainy, valy = train_test_split(X, y, test_size=0.6, random_state=2)

In [ ]:
porc_part(trainy)

In [7]:
def mi_modelo(metrica):
  model = keras.Sequential([
      keras.layers.Dense(64, activation='relu', input_shape=(trainX.shape[-1],)),
      #keras.layers.Dropout(0.5),
      keras.layers.Dense(64, activation='relu'),
      #keras.layers.Dropout(0.5),
      keras.layers.Dense(128, activation='relu'),
      #keras.layers.Dropout(0.3),
      keras.layers.Dense(1, activation='sigmoid')  #, bias_initializer='zeros')
      ])
  
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5),
              loss=keras.losses.BinaryCrossentropy(),
              metrics= metrica )
  

  return model

In [8]:
metrica = 'accuracy'

modelo1 = mi_modelo(metrica)

In [12]:
N_EPOCHS = 100
N_BATCH_SIZE = 32

In [ ]:
modelo1.summary()

In [ ]:
H = modelo1.fit( trainX, trainy,
              batch_size=N_BATCH_SIZE,
              epochs=N_EPOCHS,
              validation_data=(valX, valy), 
              verbose=2)

In [ ]:
valmetrica = "val_" + metrica

plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0,N_EPOCHS),H.history[metrica],label="train_acc")
plt.plot(np.arange(0,N_EPOCHS),H.history[valmetrica],label="val_acc")
plt.title("Training and Validation Metric")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

plt.figure()
plt.plot(np.arange(0,N_EPOCHS),H.history["loss"],label="train_loss") 
plt.plot(np.arange(0,N_EPOCHS),H.history["val_loss"],label="val_loss")
plt.title("Training and Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.show()

In [16]:
pred = modelo1.predict(valX, batch_size=N_BATCH_SIZE)
pp = pred.squeeze()
qq = np.zeros((len(valy),))
for k in range(len(pp)):
  if(pp[k]>0.5):   # Umbral - curva ROC
    qq[k]=1
  else:
    qq[k]=0


In [ ]:
print(classification_report(valy,qq, target_names=["0","1"]))

In [ ]:
confusion_matrix(qq, valy, labels=[0,1])

#CASO: NO-BALANCEADO:

In [20]:
X2, y2 = make_classification(n_samples=10000, n_classes=2, weights=(0.9,0.1), random_state=7, n_features=20)

In [ ]:
porc_part(y2)

In [22]:
trainX2, valX2, trainy2, valy2 = train_test_split(X2, y2, train_size=0.6, random_state=2)

In [ ]:
trainX2.shape

In [ ]:
porc_part(trainy2)

In [ ]:
porc_part(valy2)

Ver la siguiente liga para los pesos: class_weight:

https://www.tensorflow.org/api_docs/python/tf/keras/Model 

Así, usando dicha fórmula:


weight_for_0 = (1 / neg) * (total / 2.0)

weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}



In [ ]:
i0, i1 = Counter(trainy2).items()
print(i0)
print(i1)

In [27]:
neg = i0[1]   # total de elementos de la clase negativa
pos = i1[1]   # total de elementos de la clase positiva
total = neg + pos

In [ ]:
w_for_0 = (1/neg) * (total/2)
w_for_1 = (1/pos) * (total/2)

print(w_for_0)
print(w_for_1)

In [ ]:
N_EPOCHS = 200
N_BATCH_SIZE = 256   # revisar bien con el tamaño del val_set, que el BATCH_SIZE no quede demasiado grande

metrica2 = 'accuracy'


modelo2 = mi_modelo(metrica2)

H = modelo2.fit( trainX2, trainy2,
              batch_size=N_BATCH_SIZE,
              epochs=N_EPOCHS,
              validation_data=(valX2, valy2), 
              #class_weight=                  # {0:w_for_0, 1:w_for_1},  # {0:.1,1:.9},   # Matriz de Pesos para Clases no-balanceadas...
              verbose=2)

In [ ]:
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0,N_EPOCHS),H.history['accuracy'],label="train_acc")
plt.plot(np.arange(0,N_EPOCHS),H.history['val_accuracy'],label="val_acc")
plt.title("Training and Validation Metric")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

plt.figure()
plt.plot(np.arange(0,N_EPOCHS),H.history["loss"],label="train_loss") 
plt.plot(np.arange(0,N_EPOCHS),H.history["val_loss"],label="val_loss")
plt.title("Training and Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.show()

In [ ]:
pred = modelo2.predict(valX2, batch_size=N_BATCH_SIZE)  #.ravel()
pp = pred.squeeze()
qq = np.zeros((len(valy2),))
for k in range(len(pp)):
  if(pp[k]>0.5):
    qq[k]=1
  else:
    qq[k]=0

print(classification_report(valy2,qq, target_names=["0","1"]))

In [ ]:
cm = confusion_matrix(qq, valy2, labels=[0,1])
cm

In [ ]:
tt = cm[0,0]+cm[1,1]+cm[0,1]+cm[1,0]
print("FP: ", cm[1,0]/tt)
print("FN: ", cm[0,1]/tt)

print("\nTP_rate: ", cm[1,1]/(cm[1,1]+cm[0,1]))
print("\nFP_rate: ", cm[1,0]/(cm[0,0]+cm[1,0]))

###CURVA-ROC

In [45]:
fpr_rf, tpr_rf, thresholds_rf = roc_curve(valy2, pp)
auc_rf = auc(fpr_rf, tpr_rf)

In [ ]:
plt.figure()
plt.plot([0, 1], [0, 1], 'k--')
#plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.plot(fpr_rf, tpr_rf, label='RF (area = {:.3f})'.format(auc_rf))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

#F-beta-score

In [47]:
TN = cm[0,0]
TP = cm[1,1]
FN = cm[0,1]
FP = cm[1,0]

In [ ]:
fbeta2 = 5*TP / (5*TP+FP+4*FN)  # se pone mayor importancia a minimizar los FN
fbeta2

In [ ]:
fbeta_5 = 1.25*TP / (1.25*TP+FP+0.25*FN)   # se pone mayor importancia a minimizar los FP
fbeta_5